In [30]:
import os

import cv2
import numpy as np
import pandas as pd
from keras import Sequential
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Activation
from keras.preprocessing import image
from keras.applications import mobilenet_v2
from keras import Model

In [2]:
# ścieżka do danych zbioru treningowego
TRAIN_LABELS_FILE = r"C:\Users\HP\Documents\Python\DataAnalyst\InfoShare\DL\datasets\dresses\train\labels.txt"
# ścieżka do danych zbioru walidacyjnego
VAL_LABELS_FILE = r"C:\Users\HP\Documents\Python\DataAnalyst\InfoShare\DL\datasets\dresses\val\labels.txt"
# ścieżka do danych zbioru testowego
TEST_LABELS_FILE = r"C:\Users\HP\Documents\Python\DataAnalyst\InfoShare\DL\datasets\dresses\test\labels.txt"
#ścieżka do nazw kolorów
COLOR_NAMES_FILE = r"C:\Users\HP\Documents\Python\DataAnalyst\InfoShare\DL\datasets\dresses\names.txt"

In [3]:
#nadawanie kolumną nazwy kolorów
color = pd.read_csv(COLOR_NAMES_FILE)
color = color.T
color_list = list(color.iloc[0])
color_list.insert(0,'beige')
color_list.insert(0,'path')

In [4]:
train = pd.read_csv(TRAIN_LABELS_FILE,sep=" ",names=color_list)

In [19]:
def get_generator(filename, number=None):
    # załadowanie pliku txt do ramki pandas
    df = pd.read_csv(filename, delimiter=' ', names=color_list, dtype="str")
    if number:
        df = df[:number]
    # stworzenie generatora
    gen = image.ImageDataGenerator()
    # folder z danymi, do którego będą doklejane ścieżki z odpowiedniej kolumny
    directory = os.path.dirname(filename)
    # stworzenie iteratora po danych z zadanymi opcjami
    return gen.flow_from_dataframe(df, directory, "path", y_col = color_list[1:], target_size=(224, 224), batch_size=32,class_mode="other")

In [32]:
def create_model(gen):
    # używamy gotowego Mobilenet wytrenowanego na imaganet, ale musimy pozbyć się ostatniej warstwy klasyfikującej
    # i zastąpić ją swoją, stąd include_top=False i podany pooling
    model = mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), include_top=False, pooling="avg", weights="imagenet")
    # dodajemy na koniec sieci warstwę klasyfikującą
    dense = Dense(14,activation="softmax")(model.output)
    # budujemy nowe model, mobilenet z naszą warstwą klasyfikującą
    model = Model(inputs=[model.input], outputs=[dense])
    # szybki optymalizator adam - pozwala na szybszy trening, dobry loss do klasyfikacji 1 z k, dodatkowo patrzymy jak zmienia się accuracy
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["categorical_accuracy"])

    return model

In [8]:
def test_model(model):
    # dane testowe
    test_gen = get_generator(TEST_LABELS_FILE)

    # uruchomienie ewaluacji
    results = model.compile(test_gen)
    # wypisanie w pętli, wszytskich wartości metryk wraz z ich nazwami
    for name, result in zip(model.metrics_names, results):
        print("{}: {}".format(name, result))

In [20]:
train_gen = get_generator(TRAIN_LABELS_FILE, 200)

Found 200 validated image filenames.


In [28]:
def main():
    train_gen = get_generator(TRAIN_LABELS_FILE, 200)

    # dane walidacyjne
    val_gen = get_generator(VAL_LABELS_FILE)

    # nasze callbacki
    #callbacks = get_callbacks()

    # stwórz model
    model = create_model(train_gen)
    # ładowanie wyuczonych wcześniej wag sieci
    # model.load_weights("weights.hdf5")
    # wypisz podsumowanie modelu, wyświetla użyte warstwy i liczbę parametrów do treningu
    model.summary()
    # trening, dodajemy walidację i callbacki
    model.fit_generator(train_gen, steps_per_epoch=train_gen.n // train_gen.batch_size, epochs=10,
                        validation_data=val_gen, validation_steps=val_gen.n // val_gen.batch_size)


In [33]:
main()

Found 200 validated image filenames.
Found 449 validated image filenames.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________

Epoch 1/10
6/6 [==============================] - 506s 84s/step - loss: 3.0437 - categorical_accuracy: 0.3429 - val_loss: 2.6479 - val_categorical_accuracy: 0.4353
Epoch 2/10


KeyboardInterrupt: 